In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
df_train = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')

In [ ]:
df_train.info()
target=df_train['target']
#F27 is character values, lets try training without it.
df_train.drop(['f_27','target'],axis = 1,inplace=True)
features = df_train.iloc[:,1:31]

In [ ]:
from sklearn.model_selection import train_test_split
X=features
y=np.ravel(target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42) 

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)  

X_train.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

tf.debugging.set_log_device_placement(True)

model = Sequential()
model.add(Dense(30, activation='relu', input_shape=(30,)))
model.add(Dense(30, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
optimizer='sgd',
metrics=['accuracy'])
model.fit(X_train, y_train,epochs=5, verbose=1)

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='/tmp/model.png', show_shapes=True,)

In [ ]:
y_pred = model.predict(X_test)
y_pred
np.argmax(y_pred,axis=1)

In [ ]:
score = model.evaluate(X_test, y_test,verbose=1)
print(score)

In [ ]:
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')
df_test.drop(['f_27'],axis = 1,inplace=True)
features_test = df_test.iloc[:,1:31]
X_submissions =  scaler.transform(features_test)
y_submissions = model.predict(X_submissions)

In [ ]:
y_submissions[1][0]

In [ ]:
import csv
with open('submission.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['id', 'target'])
    for i,v in enumerate(y_submissions):
        csvwriter.writerow([i+900000,str(v[0])])